FORUM PROJECT BY TIM RODANIM NZUNGU MUZINGA

In this notebook I will explain the setup regarding my database and the queries that I will be using for my project.

This notebook will be divided in to:

++++++++++++++++++++++++++++++++++++++++++

1. Creating Tables  
2. Display of data in tables
3. Fetching Data

    3.1 User Authentication query's

    3.2 Vieuw Forums and Subforms query's

    3.3 Thread query's

    3.4 Post query's
    
    3.5 User profile query's

    3.6 Administrator
|

+++++++++++++++++++++++++++++++++++++++++



1. CREATING TABLES




CREATE TABLE IF NOT EXISTS user (  

 user_id INTEGER PRIMARY KEY AUTOINCREMENT, 

 username TEXT NOT NULL, 

 age INTEGER, 

 email TEXT, 

 password TEXT NOT NULL, 

 account_type TEXT CHECK (account_type IN ('user', 'administrator')), 

 profile TEXT, 
 
profile_image BLOB 
);

++Create Forums table with a foreign key to user (for authorship)++

CREATE TABLE IF NOT EXISTS Forums ( 

 forum_id INTEGER PRIMARY KEY AUTOINCREMENT, 

 title TEXT NOT NULL, 

 text TEXT NOT NULL, 

 user_id INTEGER, 

 FOREIGN KEY (user_id) REFERENCES user(user_id)  
);

++Create Subforms table with a foreign key to Forums++

CREATE TABLE IF NOT EXISTS Subforms (  

subform_id INTEGER PRIMARY KEY AUTOINCREMENT, 

 title TEXT NOT NULL, 

 forum_id INTEGER, 

 FOREIGN KEY (forum_id) REFERENCES Forums(forum_id)  

);

++Create Threads table with a foreign key to Subforms++

CREATE TABLE IF NOT EXISTS Threads (  

thread_id INTEGER PRIMARY KEY AUTOINCREMENT, 

 title TEXT NOT NULL, 

 subform_id INTEGER,  

user_id INTEGER, 

created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP 

 FOREIGN KEY (subform_id) REFERENCES Subforms(subform_id), 

 FOREIGN KEY (user_id) REFERENCES user(user_id) 

); 

++Create Posts table with foreign keys to Threads and Users++

CREATE TABLE IF NOT EXISTS Posts ( 

 post_id INTEGER PRIMARY KEY AUTOINCREMENT, 

 title TEXT NOT NULL, 

 post_text TEXT NOT NULL, 

 thread_id INTEGER, 

 user_id INTEGER, 

created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP, 

 FOREIGN KEY (thread_id) REFERENCES Threads(thread_id), 
 
 FOREIGN KEY (user_id) REFERENCES user(user_id) );

In [11]:
import sqlite3
conn = sqlite3.connect("Database_Reasoning.db")


DISPLAY OF DATA IN TABLES 

The above 'sql' commands have been used directly in to the sqlite workbench,

which means that in this notebook I imported a databases that was already setup.

The *db has been filled up with dummy data so that we're able to display the working and efficientcy of our query's.

In order to display all the data of a table we use the  'SELECT * FROM table' command 

(I have displayed them all in the underneath code for visualitsation of the dummy data)


*db= database

In [13]:
cur = conn.cursor()

cur.execute('SELECT * FROM user')
#cur.execute('SELECT * FROM forums')
#cur.execute('SELECT * FROM Subforms')
#cur.execute('SELECT * FROM Threads')
#cur.execute('SELECT * FROM Posts')


rows = cur.fetchall()

for row in rows:
    print(row)



(1, 'john_doe', 28, 'john@example.com', 'password123', 'user', 'Software Engineer, loves gaming', None)
(2, 'jane_smith', 34, 'jane@example.com', 'securePass!', 'administrator', 'Community Manager, passionate about art', None)
(3, 'sam_lee', 22, 'sam.lee@example.com', 'myPassword', 'user', 'Student and tech enthusiast', None)
(4, 'linda_chan', 29, 'linda.chan@example.com', 'lindasPass', 'user', 'Graphic Designer, loves traveling', None)


3.1 User Authentication query's

a. Whem a user (after registering), want to login in we will call this query to lookup the users credentials and allowing them to the home page if username and password are met.

=> SELECT * FROM user WHERE username = ? AND password = ?, ("username", "password")

b. For retrieving a user_id we will use:

=> SELECT * FROM Users WHERE user_id = ?

*password will in my app be internally hashed

In [21]:
cur.execute("SELECT * FROM user WHERE username = ? AND password = ?", ("john_doe", "password123"))

# Fetch all rows that match the query
rows = cur.fetchone()

# Check if any rows were returned
if not rows:
    print("User doesn't exist")
else:
    # Print each row if there are matching results
    for row in rows:
        print(row)

1
john_doe
28
john@example.com
password123
user
Software Engineer, loves gaming
None


3.2 Vieuw Forums and Subforms query's

a. If we would want to see what all the forums are in our table by name we'll use:

=> SELECT * FROM Forums ORDER BY title

b. If we want to see which subforums are from a certain forum than we'll use: 

=> SELECT * FROM Subforms WHERE forum_id = ? ORDER BY title

In [27]:
cur.execute("SELECT * FROM Subforms WHERE forum_id = 1 ORDER BY title")

rows = cur.fetchall()

for row in rows:
    print(row)

(2, 'Announcements', 1)
(1, 'Introductions', 1)


 3.3 Thread query's

a. Here we are getting all the threads for a specific forum ordered by most recent posts (datestamp is given uppon creation of the post)(Also a way for keeping everyting organized):

=> SELECT Threads.*, MAX(Posts.created_at) AS last_post_date

FROM Threads

LEFT JOIN Posts ON Threads.thread_id = Posts.thread_id

WHERE Threads.subform_id = ?

GROUP BY Threads.thread_id

ORDER BY last_post_date DESC

#So we are first sellecting all the columns in the table Threads and the last post made based on it's date 

#Now we retrieve all threads and eventually join them with the Posts table to find the latest post for each thread

#means that all rows from the Threads table will be included in the result,

#even if there are no corresponding rows in the Posts table. If a thread has no posts, the last_post_date will be NULL for that thread


b. When we want to create a new Thread in a certain Subform based on its ID we'll use:

=> INSERT INTO Threads (title, subform_id, user_id) VALUES (?, ?, ?)


c. When someone who has the function of administrator want's to thread by it's ID we'll use:

=> DELETE FROM Threads WHERE thread_id = ?


In [ ]:
cur.execute('''
    SELECT Threads.*, MAX(Posts.created_at) AS last_post_date

    FROM Threads

    LEFT JOIN Posts ON Threads.thread_id = Posts.thread_id

    WHERE Threads.subform_id = 1 

    GROUP BY Threads.thread_id

    ORDER BY last_post_date DESC 
''')


rows = cur.fetchall()

for row in rows:
    print(row)

(1, 'Welcome to the Forum!', 1, 2, '2024-11-01 10:15:00', '2024-11-01 10:20:00')


3.4 Post query's

a. Retrieve all posts in a thread, ordered chronologically:

=> SELECT * FROM Posts WHERE thread_id = ? ORDER BY created_at

b. Insert a new post in a thread:

=>INSERT INTO Posts (title, post_text, thread_id, user_id) VALUES (?, ?, ?, ?)


c. (Administrator): Remove a specific post by its ID:

=>DELETE FROM Posts WHERE post_id = ?



In [31]:
cur.execute('''
   SELECT * FROM Posts WHERE thread_id = 2 ORDER BY created_at 
''')


rows = cur.fetchall()

for row in rows:
    print(row)

(2, 'Follow the Rules', 'Please make sure to follow the forum guidelines.', 2, 2, '2024-11-01 11:05:00')


    3.5 User profile query's

a. Retrieve a user's profile, including profile details and recent posts.

=>SELECT * FROM user WHERE user_id = ?

b. List recent posts made by a user.

=>SELECT * FROM Posts WHERE user_id = ? ORDER BY created_at DESC LIMIT 5

*DESC LIMIT 5 = we are sorting in descending order and we only want the first 5

c. Update the profile text or profile image for a user.

=>UPDATE user SET profile = ?, profile_image = ? WHERE user_id = ?


In [ ]:
cur.execute('''
   SELECT * FROM Posts WHERE user_id = 1 ORDER BY created_at DESC LIMIT 5
''')


rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()

(3, 'Python is King', 'I think Python is the best language for beginners.', 3, 1, '2024-11-02 09:35:00')
(1, 'Hello Everyone!', 'Excited to join the community!', 1, 1, '2024-11-01 10:20:00')


    3.6 Adminstrator


These functions are all for the admininstrator and can only will only be used if the account_type == 'administrator'

a. Create a new subforum within a forum.

=>INSERT INTO Subforms (title, forum_id) VALUES (?, ?)


b. Update a Subform title.

=>UPDATE Subforms SET title = ? WHERE subform_id = ?


c. Delete a Subform by its ID.

=>DELETE FROM Subforms WHERE subform_id = ?


d. Fetching all threads and posts in a subforum, used before deleting a subforum

=>SELECT thread_id FROM Threads WHERE subform_id = ?